In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
path = "../data/online_retail_II.csv"
df = pd.read_csv(path, sep="," , encoding='latin-1')
df.head(1)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom


In [3]:
print("📊 REPORTE DE CALIDAD DE DATOS")
pd.options.display.float_format = '{:,.2f}'.format
print("="*40)
# 1. Dimensiones Básicas

print(f"📐 Dimensiones totales:")
print(f"   - Filas:    {df.shape[0]:,}")
print(f"   - Columnas: {df.shape[1]}")
# 2. Tipos de Datos y Nombres de Columnas

print("\n📋 Tipos de datos por columna:")
print(df.dtypes)
# 3. Identificación de Valores Nulos (Vacíos)

nulos = df.isnull().sum()
nulos_pct = (df.isnull().sum() / len(df)) * 100
print("\n❌ Valores Nulos (Cantidad y Porcentaje):")
print(pd.concat([nulos, nulos_pct], axis=1, keys=['Cantidad', '% Nulos']).sort_values('Cantidad', ascending=False))
# 4. Identificación de Duplicados

duplicados = df.duplicated().sum()
print(f"\nFilas totalmente duplicadas: {duplicados:,} ({(duplicados/len(df))*100:.2f}%)")
# 5. Resumen Estadístico Básico (Para detectar anomalías numéricas)

print("\n📈 Estadísticas rápidas (Ojo a los negativos en 'Min'):")
display(df.describe())

📊 REPORTE DE CALIDAD DE DATOS
📐 Dimensiones totales:
   - Filas:    1,067,371
   - Columnas: 8

📋 Tipos de datos por columna:
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

❌ Valores Nulos (Cantidad y Porcentaje):
             Cantidad  % Nulos
Customer ID    243007    22.77
Description      4382     0.41
StockCode           0     0.00
Invoice             0     0.00
Quantity            0     0.00
InvoiceDate         0     0.00
Price               0     0.00
Country             0     0.00

Filas totalmente duplicadas: 34,335 (3.22%)

📈 Estadísticas rápidas (Ojo a los negativos en 'Min'):


,Quantity,Price,Customer ID
count,"1,067,371.00","1,067,371.00","824,364.00"
mean,9.94,4.65,"15,324.64"
std,172.71,123.55,"1,697.46"
min,"-80,995.00","-53,594.36","12,346.00"
25%,1.00,1.25,"13,975.00"
50%,3.00,2.10,"15,255.00"
75%,10.00,4.15,"16,797.00"
max,"80,995.00","38,970.00","18,287.00"


In [4]:
# Limpieza
df_clean = df.copy()
codigos_extraños = ['POST', 'D', 'M', 'BANK CHARGES', 'PADS', 'DOT', 'CRUK', 'C2']
df_clean = df_clean[~df_clean['StockCode'].isin(codigos_extraños)]
df_clean = df_clean.dropna(subset=['Customer ID'])
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['Price'] > 0)]
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['Price']
df_clean.head(1)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,"13,085.00",United Kingdom,83.40


In [63]:
df_clean.describe()

,Quantity,InvoiceDate,Price,Customer ID,TotalAmount
count,"802,679.00",802679,"802,679.00","802,679.00","802,679.00"
mean,13.32,2011-01-02 08:58:01.201900544,2.94,"15,338.10",21.73
min,1.00,2009-12-01 07:45:00,0.03,"12,346.00",0.06
25%,2.00,2010-07-07 11:53:00,1.25,"13,988.00",4.95
50%,5.00,2010-12-03 14:54:00,1.95,"15,276.00",11.80
75%,12.00,2011-07-28 13:02:00,3.75,"16,809.00",19.50
max,"80,995.00",2011-12-09 12:50:00,649.50,"18,287.00","168,469.60"
std,143.86,NaN,4.40,"1,693.94",222.57


In [16]:
fact['TotalAmount'].sum()

np.float64(17438960.299999997)

In [12]:
df_raw=df_clean.copy()
df_raw['Hour_Num'] = df_raw['InvoiceDate'].dt.hour
df_raw['InvoiceDate'] = df_raw['InvoiceDate'].dt.strftime('%d-%m-%Y')
df_raw['Hour'] = df_raw['Hour_Num'].apply(lambda x: f"{x:02d}:00 - {x+1:02d}:00")
df_raw['Day_time'] = df_raw['Hour_Num'].apply(lambda x: '(1) Morning' if 6 <= x < 12 else ('(2) Afternoon' if 12 <= x < 18 else '(3) Nigth'))
df_raw.head(1)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalAmount,Hour_Num,Hour,Day_time
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,01-12-2009,6.95,"13,085.00",United Kingdom,83.40,7,07:00 - 08:00,(1) Morning


In [13]:
fact = df_raw.groupby(['StockCode','Description','InvoiceDate','Country','Price','Day_time']).agg({
    'TotalAmount': 'sum',  # Total gastado en esa factura
    'Quantity': 'sum',     # Total de artículos en la canasta
    'Invoice': 'count'
    }).reset_index()

fact.head(1)

,StockCode,Description,InvoiceDate,Country,Price,Day_time,TotalAmount,Quantity,Invoice
0,10002,INFLATABLE POLITICAL GLOBE,01-02-2010,United Kingdom,0.85,(1) Morning,2.55,3,1


In [14]:
fact.describe()

,Price,TotalAmount,Quantity,Invoice
count,"586,263.00","586,263.00","586,263.00","586,263.00"
mean,2.99,29.75,18.24,1.37
std,4.76,263.31,169.65,0.90
min,0.03,0.06,1.00,1.00
25%,1.25,6.60,3.00,1.00
50%,1.95,15.00,7.00,1.00
75%,3.75,26.25,15.00,1.00
max,649.50,"168,469.60","80,995.00",36.00


In [15]:
fact.to_csv('../data/fact.csv', sep='|', encoding='latin1', index=False)
df_raw.to_csv('../data/data_raw.csv',sep='|', encoding='latin1', index=False)